In [1]:
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using JLD
using BasicPOMCP

┌ Info: Recompiling stale cache file /Users/shengji/.julia/compiled/v1.0/AA228FinalProject/uFJfC.ji for AA228FinalProject [fe2df5ea-4d44-4e5a-a895-9dbc87b19b37]
└ @ Base loading.jl:1190

(process:4745): Gtk-WARNING **: 23:17:23.379: Locale not supported by C library.
	Using the fallback 'C' locale.


In [2]:
discretize = false;
config = 3 # 1,2, or 3;

In [3]:
sensor = Bumper()
# cut_points = [1, 2, 3, 5, 10, 20, 50]
# sensor = DiscreteLidar(cut_points);
vlist = [0, 0.5, 1, 2, 5]
omlist = [-0.5, -0.25, 0, 0.25, 0.5]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist));

In [18]:
if discretize
    num_x_pts = 30 # e.g. 50
    num_y_pts = 30 # e.g. 50
    num_th_pts = 20 # e.g. 20
    sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts);
    m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));
else
    mdp=RoombaMDP(config=config, aspace=aspace, stairs_penalty=-1000.0, goal_reward=1000.0, time_pen=-0.001, contact_pen=-1.0)
    m = RoombaPOMDP(sensor=sensor, mdp=mdp);
end;

In [19]:
num_particles = 5000
resampler = BumperResampler(num_particles)
# resampler = LidarResampler(num_particles, LowVarianceResampler(num_particles))
spf = SimpleParticleFilter(m, resampler)
v_noise_coefficient = 2.0
om_noise_coefficient = 0.5
belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [20]:
solver = POMCPSolver(max_depth=5, c=1.0, tree_queries=3000)
POMCP_planner = BasicPOMCP.solve(solver, m);

In [21]:
# is = RoombaState(-20., -15., 0., 0.)
# dist = initialstate_distribution(m)
# b0 = initialize_belief(belief_updater, dist)
Random.seed!(70)
# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
# sleep(10)
# for (t, step) in enumerate(stepthrough(m, POMCP_planner, belief_updater, b0, is, max_steps=300))
for (t, step) in enumerate(stepthrough(m, POMCP_planner, belief_updater, max_steps=1000))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

In [8]:
m.mdp.room

AA228FinalProject.Room(AA228FinalProject.Rectangle[Rectangle([-25.0 -20.0; -25.0 -5.0; -15.0 -5.0; -15.0 -20.0], Bool[true, false, true, true], AA228FinalProject.LineSegment[LineSegment([-25.0, -20.0], [-25.0, -5.0], false, false), LineSegment([-15.0, -5.0], [-15.0, -20.0], false, false), LineSegment([-25.0, -20.0], [-15.0, -20.0], false, false)], 9.0, 14.5, [-20.0, -12.5], 130.5, -24.5, -15.5, -19.5, -5.0), Rectangle([-25.0 -5.0; -25.0 5.0; -15.0 5.0; -15.0 -5.0], Bool[true, true, false, false], AA228FinalProject.LineSegment[LineSegment([-25.0, -5.0], [-25.0, 5.0], false, true), LineSegment([-25.0, 5.0], [-15.0, 5.0], false, false)], 9.5, 9.5, [-20.0, 0.0], 90.25, -24.5, -15.0, -5.0, 4.5), Rectangle([-15.0 -5.0; -15.0 5.0; 10.0 5.0; 10.0 -5.0], Bool[false, true, false, true], AA228FinalProject.LineSegment[LineSegment([-15.0, 5.0], [10.0, 5.0], false, false), LineSegment([-15.0, -5.0], [10.0, -5.0], false, false)], 25.0, 9.0, [-2.5, 0.0], 225.0, -15.0, 10.0, -4.5, 4.5), Rectangle([10.0